### Este es un cuaderno simple para construir y entrenar una Máquina de Vectores the Soporte ("Support Vector Machine," SVM) para discriminar entre dos tipos de eventos de colisión.

Acompaña al Capítulo 4 del libro.

Los datos para este ejercicio fueron generosamente proporcionados por [Sascha Caron](https://www.nikhef.nl/~scaron/).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman. Traducción a Español por Lucia Perez y Rosario Cecilio-Flores-Elie.

In [ ]:
### Primero se introduce los módulos de python pertinentes

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
### Y se le dice a pandas como mostrar los datos

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
rc('text', usetex=False)

Primero se leen las características ("featurues") y las etiquetas ("y" aquí).

In [ ]:
features = pd.read_csv('../data/ParticleID_features.csv', index_col='ID')

In [ ]:
features.head(10)

In [ ]:
features.shape

In [ ]:
y = np.genfromtxt('../data/ParticleID_labels.txt', dtype = str)

In [ ]:
y

#### Las etiquetas que indican categorías (en cadena de caracteres, "string-type") se tienen que transformar a una matriz, e.g. 0 o 1.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #cambia lo categórico a 1...N 

In [ ]:
y

In [ ]:
y = le.fit_transform(y)

In [ ]:
y  # Mosca que esto es una 1 para la primera ocasión, pero de verdad queremos que "4top" sea la etiqueta positiva.

In [ ]:
target = np.abs(y - 1)

In [ ]:
target # Mejor!

#### Examinemos estas características, usando la propiedad "describir" (describe).

In [ ]:
features.describe() # Mosca: esto automáticamente excluye las columnas que no son de tipo numérico

### ¡Importante!

Si leemos la hilera de "count" (contar), se ve que el conjunto de data completo tiene 5,000 hileras, pero no todos las columnas existen para todos. Esto es porque una colisión crea un número de productos variable.

#### Opción 1: Solo usar las primeras 16 columnas (o, los primero cuatro productos) para tener menos problemas con calculaciones y manipulación de datos.

In [ ]:
features_lim = features[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16']]

In [ ]:
features_lim.head(20)

In [ ]:
features_lim.describe() #esto automáticamente excluye las columnas que no son de tipo numérico, y no cuenta valores ausentes y "NaNs" (no un número)

¡Todavia hay columnas de característica de diferentes largos! Pueden existir valores NaN. Por ahora, se reemplazan con 0.

In [ ]:
# Examinemos la columna P10

np.where(np.isnan(features_lim.P10))

In [ ]:
# Se mete un 0 donde este un NaN

features_lim = features_lim.fillna(0) 

#### ¿Que dará "describe" ahora?

In [ ]:
features_lim.describe()

Muy bien - tenemos tamaños consistentes, y ahora podemos usarlos como matrices de caracteristica. PERO, seamos conscientes de los impactos negativos que pueden venir de estas estrategias de entrada.

### Revisión de aprendizaje

P: ¿Qué hace el método "describe" de pandas, y que nos cuenta de un marco de datos?
    
<details>
    <summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
    <p>
         El método "describe" da la información y estadísticas útiles de los datos en el marco. Imprime el número total de objetos en el marco de datos, y otra información como: el promedio, la frecuencia, el mínimo, el máximo, y más. Mosca que solo mostrará esta información si aplica al tipo de dato en el marco: solo incluyera columnas numéricas.
    </p>
</details>

### Rápidamente, exploremos las etiquetas.

In [ ]:
np.sum(target)/len(target) #la distribución (ayuda a crear un punto de referencia)

84% de etiquetas con negativos, 16% positivas. Esto en un poco desbalanceado; así, un clasificador que pone todo en la clase negativa tendrá una exactitud de 84%.

### ¿Porque no usamos un clasificador aleatorio, que asigna una clase aleatoria según la distribución de clases?

In [ ]:
# Solución numérica

acc=0
for i in range(1000):
    x = np.random.choice(target,5000)
    acc += metrics.accuracy_score(target,x)
print(acc/1000)


# Solución analítica

print(0.8378*(0.8378) + 0.1622*0.1622)

### Empecemos con un modelo lineal: model = SVC()

Establecemos un punto de referencia: el modelo lineal, sin regularización (o, el C parámetro muy alto)

In [ ]:
bmodel = LinearSVC(dual = False, C = 1000) # se prefiere "dual = False" cuando el número de muestras es más grande que el número de características. ¡Si no, no se convergerá!

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=101) 

In [ ]:
l_benchmark_lim = cross_validate(bmodel, features_lim, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
l_benchmark_lim

In [ ]:
np.round(l_benchmark_lim['test_score'].mean(),3), np.round(l_benchmark_lim['test_score'].std(), 3)

También es bueno revisar las etiquetas predichas. "Cross\_val\_predict" reunirá las etiquetas predichas cuando cada objeto estaba en el pliegue de prueba.

In [ ]:
ypred_bench_lim = cross_val_predict(bmodel, features_lim, target, cv = cv)

Es un poco mejor que un clasificador aleatorio, pero peor que el clasificador bobo que le dice "¡no!" a todo.

### ¿Y si escalamos los valores?

In [ ]:
from sklearn.pipeline import make_pipeline # Con esto, se puede construir varias etapas a la vez

In [ ]:
piped_model = make_pipeline(StandardScaler(), LinearSVC(dual = False, C = 1000)) #cambianos a SVC lineal

benchmark_lim_piped = cross_validate(piped_model, features_lim, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark_lim_piped

In [ ]:
np.round(benchmark_lim_piped['test_score'].mean(),3), np.round(benchmark_lim_piped['test_score'].std(), 3)

Se mejoró mucho, y aprendemos algo de lo que nos está dando problemas cuando comparamos las notas de entrenamiento y prueba. La manera formal de investigar esto es con las curvas de aprendizaje, que muestran la brecha entre las notas de entrenamiento y prueba, y también si se necesitan más datos.

### Las curvas de aprendizaje

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy', scale = False):
    """
    Generate a simple plot of the test and training learning curve.

    Producir una gráfica simple de la curva de prueba y aprendizaje.

    parámetros
    ----------
    estimator : (estimador) el tipo de objeto que implementa los métodos de "fit" (encajar) y "predict" (predecir).
        Este tipo de objeto es el que se clona para cada validación. CONFIRM MEANING!!!
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    
    title : cadena de caracteres
        Título para la gráfica
    
    X : en forma de matriz, con forma de (n_samples, n_features)
        El vector de entrenamiento, donde n_samples es el número de muestras, y n_features es el número de características

    y : en forma de matriz, con forma de (n_samples) o (n_samples, n_features); opcional
        Meta relacionada a X para clasificación o regresión; usar "None" (ninguno) para aprendizaje 

    ylim : tupla, con forma de  (ymin, ymax), opcional
        Define los valores mímimos y máximos de valores-y trazados.

    cv : entero, generador de entre-validación o un iterable, opcional
        Determina la estrategia de entre-validación con divisiones.
        Entradas posibles para cv incluyen:
          - None (ninguno), para usar la entre-validación de 3 pliegues (por defecto),
          - entero, para precisar cuantos pliegues,
          - :term:`CV splitter (entre-validación separador)`,
          - un iterable que rinde divisiones de (entrenamiento, prueba) como matrices de índices.

        Para entradas de enteros o None, si ``y`` es binario o de varias clases, se utiliza :class:`StratifiedKFold`.
        Si el estimador no es clasificador, o si ``y`` no es binario o de varias clases, se utiliza :class:`KFold`.

        Consultar a :ref:`User Guide <cross_validation>` para aprender de los diferentes entre-validadores que se pueden usar aquí.

    n_jobs : entero o None, optional (por defecto = None)
        El número de trabajos que operarán en paralelo.
         ``None`` significa 1 a menos si en el contexto de :obj:`joblib.parallel_backend`.
        ``-1`` significa que usará todos los procesadores. Consultar a  :term:`Glossary <n_jobs>` para mas detalles.

    train_sizes : en forma de matriz, con forma de (n_ticks,), dtype float or int
        Los números de ejemplos de entrenamiento (relativo o absolutos) que se usarán en generar la curva de aprendizaje.
        Si el dtype es float, train_sizes se usa como la fracción del tamaño máximo del conjunto de prueba (el cual se determina
        con el método de validación actual); o, tiene que ser entre (0,1]. Si no, train_sizes se interpreta como los tamaños
        absolutos de los conjuntos de entrenamiento. Mosca: para clasificación, el número de pruebas usualmente tiene que 
        ser suficiente para tener por lo menos una muestra de cada clase. (Por defecto: np.linspace(0.1, 1.0, 5))

    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("núm. de ejemplos de entrenamiento",fontsize = 14)

    plt.ylabel("Nota de Exactitud",fontsize = 14)

    if (scale == True):
        scaler = sklearn.preprocessing.StandardScaler()
        X = scaler.fit_transform(X)

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="b")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="b",
             label="Nota de entrenamiento de entre-validación")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Nota de prueba de entre-validación")

    plt.legend(loc="best",fontsize = 12)
    return plt

In [ ]:
#plot_learning_curve(piped_model, 'Generalized Learning Curves, linear SVC model, no reg', features_lim, target, train_sizes = np.array([0.05,0.1,0.2,0.5,1.0]), cv = KFold(n_splits=5, shuffle=True))
plot_learning_curve(piped_model, 'Curvas de aprendizaje generalizado, modelo lineal de SVC, sin reg.', features_lim, target, train_sizes = np.array([0.05,0.1,0.2,0.5,1.0]), cv = KFold(n_splits=5, shuffle=True))

### Revisión de aprendizaje

P: Usando las curvas de aprendizaje, ¿que puede ser el problema, y como se puede mejorar el modelo?
<details>
    <summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
    <p>
        Este modelo padece de gran preferencia. Se nota en las curvas de aprendizaje, que muestran una brecha muy pequeña (y no importante estadísticamente) entre las notas de entrenamiento y prueba, para el tamaño actual de muestras (n = 4000). Esto por lo menos excluye a el problema de alta divergencia, y podemos enfocarnos en soluciones para gran preferencia. 
    </p>
</details>

<br/>

P: ¿Nos ayudaría tener más datos? ¿Por qué sí o no?  

<details>
    <summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
    <p>
        No nos ayudaría. Más datos aumentaría la gráfica a la derecha, con más de 4000 muestras. Pero, las curvas de aprendizaje ya se estabilizaron
        (ya se ven planas). Entonces, teniendo más datos no mejoraría las notas ni arreglaría el problema de gran preferencia.
    </p>
</details>

### Optimización de Parámetros 

(mosca: esto NO es entre-validación anidada)

In [ ]:
piped_model = make_pipeline(StandardScaler(), SVC()) #no lineal para poder cambiar el kernel

piped_model.get_params() # esto muestra como acceder parámetros para el escalador y el clasificador

### Se puede definir un cuadro de valores de parámetros para hacer la optimización. 

(Para estimar el error de generalización, se debe hacer una entre-validación anidada)

Esto puede ser lento (tomo ~5 minutos en mi laptop, pero fue 15 en la previa); modelos más complejos (especialmente esos con alto gamma) toman más tiempo.

In [ ]:
#optimizing SVC: THIS IS NOT YET NESTED CV
# optimizar SVC: ¡todavía no estamos en entre-validación anidada!

parameters = {'svc__kernel':['poly', 'rbf'], \
              'svc__gamma':[0.00001,'scale', 0.01, 0.1], 'svc__C':[0.1, 1.0, 10.0, 100.0, 1000], \
              'svc__degree': [2, 4, 8]}

model = GridSearchCV(piped_model, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(features_lim,target)

print('Mejors parámetros, y mejor nota:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

#### Visualizar las notas en un marc de datos, y ordernar por nota de prueba.

A mí me gusta ver el promedio, el std (desviación estándar), el preoedio de las notas de entrenamiento (por si acaso son muy diferentes, y para entender lo que significa el resultado), y también el tiempo total de ajuste (queremos escoger el modelo más rápido si todo lo otro es igual).

In [ ]:
scores_lim = pd.DataFrame(model.cv_results_)

scores_lim[['params','mean_test_score','std_test_score','mean_train_score', \
            'mean_fit_time']].sort_values(by = 'mean_test_score', ascending = False)

#### También podemos aislar y estudiar un tipo de kernel a la vez.

In [ ]:
scores_lim[scores_lim['param_svc__kernel'] == 'poly'][['params','mean_test_score','std_test_score',\
                        'mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', ascending = False)

In [ ]:
scores_lim.columns

### Diagnóstico final

El problema aquí es la preferencia muy alta, que no me sorprende porque solo estamos usando parte de las características.

Podemos intentar dos cosas: inventar nuevas características que mejorarán los valores (usando lo que sabemos del problema físico), y utilizar una estrategia de entrar datos que incluye información de características desechadas.

### Próximo paso: definir nuevas características. 

In [ ]:
features = features.fillna(0) # para excluir NaN

In [ ]:
features = features.replace('', 0) # para remplazar cadenas sin ningún caráceter con 0 

In [ ]:
features.head()

In [ ]:
np.unique(features.Type_1.values)

Primero se puede ver que tipos de partículas tenemos después de la colisión.

In [ ]:
np.unique(np.array([features['Type_'+str(i)].values for i in range(1,14)]).astype('str'))

Estas son las nuevas características sugeridas (la justificación se puede ver en Capítulo 4 del libro)
    
    1. número total de partículas producidas
    2. número total de b jets
    3. número total de jets (chorros)
    4. número total de leptons (électrons, positron, mu+, mu-)

In [ ]:
# contar el número de tipos no zero

ntot = np.array([-(np.sum(np.array([features['Type_'+str(i)].values[j] == 0 for i in range(1,14)])) - 13) for j in range(features.shape[0])])

In [ ]:
# definir una columna nueva en el marco de datos

features['Total_products'] = ntot

In [ ]:
# contar el número de b jets

nbtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'b' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
# definir una columna nueva en el marco de datos

features['Total_b'] = nbtot

In [ ]:
# Para esta, mejor contar todo los tipos (jets, photons g, e-, e+, mu-, mu+)

njtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'j' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
ngtot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'g' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_el_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'e-' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_pos_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'e+' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_muneg_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'm-' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_mupos_tot = np.array([np.sum(np.array([features['Type_'+str(i)].values[j] == 'm+' for i in range(1,14)])) for j in range(features.shape[0])])

In [ ]:
n_lepton_tot = n_el_tot + n_pos_tot + n_muneg_tot + n_mupos_tot

Y así se definen las otras nuevas características:

In [ ]:
features['Total_j'] = njtot
features['Total_g'] = ngtot
features['Total_leptons'] = n_lepton_tot

### Revisión de aprendizaje

¿Con cual método se puede ver las primeras filas de nuestro marco de características? <i>Prueba tu código en la próxima celda.</i>


In [ ]:
# Escribe tu código aquí


<details>
    <summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```python
features.head()
```
    
</p>
</details>

### ingeniería de características 1: el impacto de impact variables ad-hoc

In [ ]:
features_lim_2 = features[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16','Total_products', 'Total_b' ,'Total_j','Total_g', 
              'Total_leptons']]

In [ ]:
bmodel # ¿te acuerdas de nuestro modelo de referencia?

In [ ]:
piped_model = make_pipeline(StandardScaler(), LinearSVC(dual = False, C = 1000))

In [ ]:
benchmark_lim2_piped = cross_validate(piped_model, features_lim_2, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark_lim2_piped

In [ ]:
np.round(benchmark_lim2_piped['test_score'].mean(),3), np.round(benchmark_lim2_piped['test_score'].std(), 3)

In [ ]:
piped_model = make_pipeline(StandardScaler(), SVC())

Este modelo se puede optimizar también; tomará tiempo, como antes.

In [ ]:
# optimizar SVC: toma tiempooooo

parameters = {'svc__kernel':['poly', 'rbf'], \
              'svc__gamma':[0.00001,'scale', 0.01, 0.1], 'svc__C':[0.1, 1.0, 10.0, 100.0], 'svc__degree': [2, 4, 8]}

nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(piped_model, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(features_lim_2,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores_lim_2 = pd.DataFrame(model.cv_results_)
scores_lim_2[['params','mean_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)

### Otro tipo de ingeniería de característica que se puede intentar es usar el tipo de producto en el sitio <i>i</i> como una característica.

Se puede hacer codificando la etiqueta, pero esto introduce la idea de una métrica de distancia (etiquetas mapeadas a 0 y 1 se interpretan a ser más cercanas a ellas mismas, que etiquetas mapeadas entre 0 y 7.)

Podemos agregar nuevas columnas para cada etiqueta de categoría, y usar 0/1 para indicar que la partícula es de ese tipo.

In [ ]:
features_add = pd.get_dummies(data=features, columns=['Type_'+str(i) for i in range(1,14)])

In [ ]:
features_add.columns[58:80]

In [ ]:
features_add.shape

### ingeniería de características 1: agregar otros variables (tipo de producto)

In [ ]:
features_lim_3 = features_add[['MET', 'METphi', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12',  'P13', 'P14', 'P15', 'P16','Total_products', 'Total_b' ,'Total_j','Total_g', 
              'Total_leptons','Type_1_b',
       'Type_1_j', 'Type_2_0', 'Type_2_b', 'Type_2_e+', 'Type_2_e-',
       'Type_2_g', 'Type_2_j', 'Type_2_m+', 'Type_2_m-', 'Type_3_0',
       'Type_3_b', 'Type_3_e+', 'Type_3_e-', 'Type_3_g', 'Type_3_j',
       'Type_3_m+', 'Type_3_m-', 'Type_4_0', 'Type_4_b', 'Type_4_e+',
       'Type_4_e-', 'Type_4_g', 'Type_4_j', 'Type_4_m+', 'Type_4_m-']]

In [ ]:
features_lim_3.head()

In [ ]:
piped_model = make_pipeline(StandardScaler(), LinearSVC(dual = False, C = 10**3))

In [ ]:
benchmark = cross_validate(piped_model, features_lim_3, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark

In [ ]:
np.round(benchmark['test_score'].mean(),3), np.round(benchmark['test_score'].std(), 3)

In [ ]:
np.round(benchmark['train_score'].mean(),3), np.round(benchmark['train_score'].std(), 3)

#### No se ve que se mejoró, pero debemos optimizar el modelo.

In [ ]:
piped_model = make_pipeline(StandardScaler(), SVC())

In [ ]:
# optimizar SVC: 

parameters = {'svc__kernel':['poly', 'rbf'], \
              'svc__gamma':[0.00001,'scale', 0.01, 0.1], 'svc__C':[0.1, 1.0, 10.0, 100.0, 1000.0], 'svc__degree': [4]} #poly never helps
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(piped_model, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(features_lim_3,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

scores_lim_3 = pd.DataFrame(model.cv_results_)
scores_lim_3[['params','mean_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)

### Por fin, podemos intentarlo con todas las características.

In [ ]:
features_add.shape

In [ ]:
piped_model = make_pipeline(StandardScaler(), LinearSVC(dual = False, C = 1000))

In [ ]:
cv

In [ ]:
benchmark = cross_validate(piped_model, features_add, target, cv = cv, scoring = 'accuracy', return_train_score=True)

In [ ]:
benchmark

In [ ]:
np.round(benchmark['test_score'].mean(),3), np.round(benchmark['test_score'].std(), 3)

In [ ]:
np.round(benchmark['train_score'].mean(),3), np.round(benchmark['train_score'].std(), 3)

### Revisión de aprendizaje

P: Con todos estos cambios y nuevos puntos de referencia, ¿que se nota de nuestro modelo? ¿Tiene todavía gran preferencia?
<details>
    <summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
Se ve que el modelo no tiene más rasgos de gran preferencia, ¡pero ahora hay una alta divergencía! Esto no me sorprende, y lo esperaba porque nuestros datos tienen mucho ruido cuando se usan todas las características.

Es posible rehacer la optimización, pero es probable que no ayudará, teniendo en cuenta todo lo que hemos hecho.
    
</p>
</details>

In [ ]:
piped_model = make_pipeline(StandardScaler(), SVC())

In [ ]:
# optimizar SVC: ¡todavía no estamos en entre-validación anidada!

parameters = {'svc__kernel':['poly', 'rbf'], \
              'svc__gamma':[0.00001, 0.001, 0.01, 0.1], 'svc__C':[0.1, 1.0, 10.0, 1000.0], 'svc__degree': [4]} #poly never helps
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(piped_model, parameters, cv = StratifiedKFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(features_add,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores_all = pd.DataFrame(model.cv_results_)
scores_all[['params','mean_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)

### La moraleja del cuento: ingeniería de características funciona a lo mejor si se usa conocimiento del sujeto, y usando más características no siempe ayuda.